In [127]:
#!pip install ipynb

In [128]:
import os
print(os.getcwd())

C:\Users\mavul\Documents\known\yahoo


In [129]:
import ipynb
from ipynb.fs.full.swagger_configure import configure_swagger
from ipynb.fs.full.rabbitmq_config import send_to_queue


## Database connection  
##### 1) load .env file
##### 2) connection string
##### 3) trial code-> to check connection

In [131]:
from sqlalchemy import create_engine,text
import urllib.parse
from dotenv import load_dotenv
load_dotenv()
server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
password = urllib.parse.quote_plus(password)
driver = 'ODBC+Driver+17+for+SQL+Server'
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes'
print(connection_string)
engine = create_engine(connection_string)


######################################################################################################################
userId=1
query = """SELECT TOP (1000) [ticker_id], [symbol], [name], [last_updated], [created_at]
        FROM [yahoo].[dbo].[Ticker_t]
        WHERE [ticker_id] = :userId"""
       #send_to_queue("user_queue", query)  # Send the query to RabbitMQ
with engine.connect() as connection:
    result = connection.execute(text(query),{"userId":userId})
    for row in result:
        print(row)

mssql+pyodbc://DESKTOP-03OG7NU/yahoo?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
(1, 'appl', 'apple     ', datetime.datetime(2024, 12, 21, 0, 0), datetime.datetime(2024, 12, 21, 0, 0))


## Flask API

In [133]:
from flask import Flask, jsonify, request
from flask_restx import Resource, fields

# Create Flask app
app = Flask(__name__)

# Configure Swagger using the imported function from swagger_config.ipynb
api = configure_swagger(app)

## data_model_swagger

In [135]:
# Define the Ticker model for Swagger documentation
ticker_model = api.model('Ticker', {
    'ticker_id': fields.Integer(required=True, description='The Ticker ID'),
    'symbol': fields.String(required=True, description='The ticker symbol'),
    'name': fields.String(required=True, description='The ticker name')
})
message_model = api.model('Message', {
    'message': fields.String(required=True, description='The message to be sent to RabbitMQ')
})
user_fetch_model = api.model('user_fetch_model', {
    'UserID': fields.Integer(required=True, description='The User ID'),
    'FirstName': fields.String(required=True, description='The First Name of the User'),
    'LastName': fields.String(required=True, description='The Last Name of the User'),
    'Email': fields.String(required=True, description='The Email of the User'),
    'Email2': fields.String(required=True, description='The Email of the User'),
    'DateOfBirth': fields.String(required=True, description='The Date of Birth of the User')
})

## Get APIs

In [137]:
# Ticker resource (GET request for ticker info from database with ticker id)
@api.route('/ticker/<int:id>')
@api.response(404, 'Ticker not found')
class Ticker(Resource):
    @api.marshal_with(ticker_model)
    def get(self, id):
        query = """
        SELECT TOP (1000) [ticker_id], [symbol], [name], [last_updated], [created_at]
        FROM [yahoo].[dbo].[Ticker_t]
        WHERE [ticker_id] = :id
        """
        with engine.connect() as connection:
            result = connection.execute(text(query), {"id": id})
            tickers = [dict(zip(result.keys(),row )) for row in result]
        
        # If ticker found, return the result
        if len(tickers) > 0:
            return tickers[0]
        else:
            api.abort(404, "Ticker not found")
            
@api.route('/user_fetch/<int:userId>')
@api.response(404, 'user not found')
class user_fetching(Resource):
    print(Resource)
    @api.marshal_with(user_fetch_model)
    def get(self, userId):
        print(userId)
        #query = "SELECT TOP (1000) [UserID], [FirstName], [LastName], [Email], [DateOfBirth] FROM [yahoo].[dbo].[Users] where [UserID]= :userId"
        query2 = """
        SELECT TOP (1000) [UserID], [FirstName], [LastName], [Email], [DateOfBirth]
        FROM [yahoo].[dbo].[Users]
        WHERE [UserID] = :userId
        """
        
        #send_to_queue("user_queue", query)  # Send the query to RabbitMQ
        with engine.connect() as connection:
            result = connection.execute(text(query2),{"userId": userId})
            users = [dict(zip(result.keys(), row)) for row in result]
            print(users)
            if users:
                return users
            else:
                api.abort(404, " not found in the database ")

<class 'flask_restx.resource.Resource'>


## POST APIs

In [139]:
# Route to send a message to RabbitMQ
@api.route('/send_message')
class SendMessage(Resource):
    @api.expect(message_model)
    @api.response(200, 'Message successfully sent')
    @api.response(400, 'No message provided')
    def post(self):
        """
        Sends a message to RabbitMQ.
        """
        message = request.json.get('message', '')
        if message:
            # Send the message to RabbitMQ
            send_to_queue(message)
            return {"status": "Message sent to RabbitMQ!"}, 200
        return {"error": "No message provided"}, 400


In [140]:
# main_app.ipynb
#from ipynb.fs.full.swagger_configure import configure_swagger  # Import the function from the notebook
#from rabbitmq_config import send_to_queue
# Run Flask app
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:38] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2024 14:28:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2024 14:28:42] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:42] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:42] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:42] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Dec/2024 14:28:

In [141]:


# Function that sends a message to the queue
def send_to_queue(queue_name, message):
    # Connect to RabbitMQ and send the message
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()

    # Ensure the queue exists before sending a message
    channel.queue_declare(queue=queue_name, durable=True)

    # Send the message
    channel.basic_publish(
        exchange='',
        routing_key=queue_name,
        body=message,
        properties=pika.BasicProperties(
            delivery_mode=2,  # Make the message persistent
        )
    )

    print(f"Sent message to queue '{queue_name}': {message}")
    connection.close()

# Example usage of the function to create a queue and send a message
#create_queue_in_rabbitmq('user_queue')  # This will automatically create the queue
#send_to_queue('user_queue', 'Test message to the queue')
